In [15]:
import json
import folium
import pandas
import geopandas

skuSetName = "gingernut"
storeResults = json.load(open(f"{skuSetName}Coles5.json"))
skus = json.load(open(f"{skuSetName}Sku.json"))

storesDataFrame = pandas.DataFrame.from_dict(storeResults.values())

# Reorders columns
cols = ["name","id","address","latitude","longitude"]
storesDataFrame = storesDataFrame[cols + [c for c in storesDataFrame.columns if c not in cols]]

storesGeoDataFrame = geopandas.GeoDataFrame(
    storesDataFrame, geometry=geopandas.points_from_xy(storesDataFrame.longitude, storesDataFrame.latitude), crs="WGS84"
)

# Okay, I really hate this. But I don't think folium/branca has a better way to do this
# This renames the SKU columns and returns a legend for each SKU
def updateSkus(skus, storesGeoDataFrame):
    html = ""
    for sku in skus.values():
        html += f'<p>{sku["svg"]}&emsp;{sku["display-name"]}</p>'
        storesGeoDataFrame.rename(columns={str(sku["id"]): str(sku["display-name"])}, inplace=True)
    return html

legend = folium.Element(f"""
    <div style="
        position: fixed;
        bottom: 50px;
        left: 50px;
        padding: 5px 40px 0px 10px;
        z-index: 9999;
        font-size: 14px;
        background-color: #ffffff;
        opacity: 0.9;">
        {updateSkus(skus, storesGeoDataFrame)}
    </div>
""")

map = folium.Map(tiles="Cartodb Positron")

for sku in skus.values():
    storesGeoDataFrame[
        storesGeoDataFrame[sku["display-name"]] != "Not Stocked"
    ].explore(
        m=map,
        tooltip= True,
        marker_type="marker",
        marker_kwds=dict(
            icon=folium.DivIcon(html=sku["svg"])),
        )

map.fit_bounds(map.get_bounds(), padding=(30,30))
map.get_root().html.add_child(legend)

map.save(f"{skuSetName}Analysis.html")
map


In [117]:
map.save("test.html")